## Problem formulation

We imagine a regression problem in $D$ dimensions with inputs $\mathbf{x}_i \in \mathbb{R}^{D}$ and one-dimensional outputs $y_i \in \mathbb{R}$ on the form of

$$ y_i = f(\mathbf{x}_i) + \epsilon(\mathbf{x}_i) $$

where $\epsilon(\mathbf{x}_i) = \mathcal{N}(0,\sigma_{\epsilon}^2(\mathbf{x}_i))$ is a noise term. If the variance $\sigma^2(\mathbf{x}_i) = \sigma^2_{\epsilon}$ is constant w.r.t. $\mathbf{x}$ it is called *homoscedastic* noise and *heteroscedastic* if not. 



Our goal is to learn a probabilistic regression model $p_{\theta}(y|\mathbf{x})$, which for now we will assume is a normal distribution:

$$p_{\theta}(y|\mathbf{x}) = \mathcal{N}(y|\mu(\mathbf{x}),\sigma_p^2(\mathbf{x}))$$

such as the case for Gaussian Processes (GPs) with predictive variance $\sigma_p^2(\mathbf{x})$. We want the model not only to have a good prediction error (e.g. a low mean squared error) but also has well-calibrated uncertainties $\sigma_p^2(\mathbf{x})$. We shall assume, the predictive variance has a (either analytical or approximal)  decomposition:

$$\sigma_p^2(\mathbf{x}) = \sigma_e^2(\mathbf{x}) + \sigma_a^2(\mathbf{x})$$

where $\sigma_a^2(\mathbf{x})$ is the *aleatoric* noise (irreducible inherent randomness in the data-generating process) and $\sigma_e^2(\mathbf{x})$ is the *epidemestic* noise (lack of knowledge, i.e. it can be reduced with better models or more data). Although the predictive variance from the GP has this exact explicit decomposition: 
$$\sigma_p^2(\mathbf{x})=\underbrace{K(\mathbf{x}_*,\mathbf{x}_*)-K(\mathbf{x}_*,\mathbf{X})(K(\mathbf{X},\mathbf{X})+\sigma_nI)^{-1}K(\mathbf{X},\mathbf{x}_*)}_{\sigma^2_e} + \underbrace{\sigma_n^2I}_{\sigma^2_a}$$

and we are guarenteed that $\sigma_n^2 \xrightarrow{} \sigma_{\epsilon}^2$ in the limit of infinite samples, no such guarentee exists for finite samples and GPs are one of the few models to have this explicit decomposition. However, in this paper we argue that if a regressor has an explicit bias-variance decomposition we can at least approximate both aleatoric and epidemistic noise terms. 

### Bias-variance decomposition
An often used loss function for regression models is the mean-squared-error (MSE) loss. With a model producing a prediction $\hat{y}$ the expectation of MSE between $\hat{y}$ and the observed $y$ can be decomposed (for brevity we have left out the dependence on input $\mathbf{x}$):
\begin{split}
\mathbb{E}[(y-\hat{y})^2] &= E[(f+\epsilon-\hat{y})^2] \\
&= E[(f-\hat{y})^2+2(f-\hat{y})\epsilon+\epsilon^2]\\
&= E[(f-\hat{y})^2]+2E[(f-\hat{y})\epsilon]+E[\epsilon^2]\\
&= E\left[\left(f - E(\hat{y}) + E(\hat{y})-\hat{y} \right)^2 \right] + 2E[(f-\hat{y})\epsilon]+\sigma_{\epsilon}^2 \\
& = Var(\hat{y}) + \text{Bias}^2(\hat{y}) + \sigma_{\epsilon}^2.
\end{split}



In [ ]:
from src.parameters import Parameters
from src.experiment import Experiment
from imports.general import *
from imports.ml import *

parameters = Parameters({"surrogate":"RF",
"experiment":"1",
"acquisition":"EI",
"data_name":"benchmark",
"n_evals":40,
"n_test":1000,
"n_validation":100,
"snr":100,
"xi":0.0,
"recalibrate":False,
"bo":True,
"extensive_metrics":True,
"d":3,
"recal_mode":"cv",
"problem_idx":5, 
"seed":0 ,
"n_initial":10},mkdir=True)
experiment = Experiment(parameters)
experiment.run()

In [ ]:
if parameters.d == 1:
	idx = np.argsort(experiment.dataset.data.X_test.squeeze())
	X_test = experiment.dataset.data.X_test[idx]
	y_test = experiment.dataset.data.y_test[idx]
	mu,sigma = experiment.optimizer.surrogate_object.predict(X_test)
	mu = mu.squeeze()
	sigma = 2*sigma.squeeze()
	fig = plt.figure()
	plt.plot(X_test, mu, marker=".")
	plt.fill_between(X_test.squeeze(), mu-sigma,mu+sigma,alpha=0.2)
	plt.plot(X_test, y_test,"x",alpha=0.2)
	plt.plot(experiment.dataset.data.X_train, experiment.dataset.data.y_train,"o")

	fig = plt.figure()
	a_vals = experiment.optimizer.acquisition_function(torch.from_numpy(np.expand_dims(X_test, 1))).cpu().detach().squeeze()
	plt.plot(X_test,a_vals)

# 1. Extensive investigation

In [ ]:
# Later when data should be loaded:
cnx = sqlite3.connect('./results.db')
print("Surrogate | cal | regret , regret_hat | tot_regret , tot_regret_hat")
for surrogate in ["BNN","GP","RF","DE"]:
	df = pd.read_sql(f"select * from {folders[0]} where bo='1' and recalibrate='0' and surrogate='{surrogate}'", cnx)
	cal_mu, cal_sigma = np.mean(df[["y_calibration_mse"]].to_numpy()),np.std(df[["y_calibration_mse"]].to_numpy())

	regret_mu, regret_sigma = np.mean(df[["f_regret"]].to_numpy()),np.std(df[["f_regret"]].to_numpy())
	tot_regret_mu, tot_regret_sigma = np.mean(df[["f_regret_total"]].to_numpy()),np.std(df[["f_regret_total"]].to_numpy())
	
	df = pd.read_sql(f"select * from {folders[0]} where bo='1' and recalibrate='1' and surrogate='{surrogate}'", cnx)
	regret_r_mu, regret_r_sigma = np.mean(df[["f_regret"]].to_numpy()),np.std(df[["f_regret"]].to_numpy())
	tot_regret_r_mu, tot_regret_r_sigma = np.mean(df[["f_regret_total"]].to_numpy()),np.std(df[["f_regret_total"]].to_numpy())
	
	print(f"{surrogate} | {cal_mu:.2E} +/- {cal_sigma:.2E} | {regret_mu:.2E} +/- {regret_sigma:.2E} , {regret_r_mu:.2E} +/- {regret_r_sigma:.2E} | {tot_regret_mu:.2E} +/- {tot_regret_sigma:.2E} , {tot_regret_r_mu:.2E} +/- {tot_regret_r_sigma:.2E}")

In [ ]:
from figs.scripts.loader import Loader
from imports.general import *
from imports.ml import *
loader = Loader()
loader.path2sql(["results_change_std"])

In [2]:
from figs.scripts.loader import Loader
from imports.general import *
from imports.ml import *
loader = Loader()

In [ ]:


# for recal in [False,True]:
#     for group in groups_:
#         out = "f_regret"
#         r_f = pd.read_sql(loader.dict2query(,table_name=table_name, columns=[out]))[[out]].to_numpy()

#         tr_f = pd.read_sql(f"select * from {folders[0]} where bo='1' and recalibrate='0' and surrogate='{sur}'  and snr='{snr}'", cnx)[["f_regret_total"]].to_numpy()
#         c_r = pd.read_sql(f"select * from {folders[0]} where bo='0' and recalibrate='0' and surrogate='{sur}'  and snr='{snr}'", cnx)[["y_calibration_mse"]].to_numpy()
#         c_bo = pd.read_sql(f"select * from {folders[0]} where bo='1' and recalibrate='0' and surrogate='{sur}'  and snr='{snr}'", cnx)[["y_calibration_mse"]].to_numpy()

#         c_r_r = pd.read_sql(f"select * from {folders[0]} where bo='0' and recalibrate='1' and surrogate='{sur}'  and snr='{snr}'", cnx)[["y_calibration_mse"]].to_numpy()
#         c_bo_r = pd.read_sql(f"select * from {folders[0]} where bo='1' and recalibrate='1' and surrogate='{sur}'  and snr='{snr}'", cnx)[["y_calibration_mse"]].to_numpy()
#         r_f_r = pd.read_sql(f"select * from {folders[0]} where bo='1' and recalibrate='1' and surrogate='{sur}'  and snr='{snr}' ", cnx)[["f_regret"]].to_numpy()
#         tr_f_r = pd.read_sql(f"select * from {folders[0]} where bo='1' and recalibrate='1' and surrogate='{sur}'  and snr='{snr}' ", cnx)[["f_regret_total"]].to_numpy()

#         row = f"{sur} " +(
#                 f"&$"
#                 + format_num(np.nanmean(c_bo_r))
#                 + "\,\,(\pm "
#                 + format_num(np.nanstd(c_bo_r))
#                 + ")$"
#             )
#     if False and sur not in ["RS", "DS"]:
        # row += (
        #     "&$"
        #     + format_num(np.nanmean(c_r))
        #     + "\,\,(\pm "
        #     + format_num(np.nanstd(c_r))
        #     + ")$"
        # )
        # row += (
        #     "&$"
        #     + format_num(np.nanmean(c_bo))
        #     + "\,\,(\pm "
        #     + format_num(np.nanstd(c_bo))
        #     + ")$"
        # )
    #     row += (
    #         f"&$"
    #         + format_num(np.nanmean(r_f))
    #         + "\,\,(\pm "
    #         + format_num(np.nanstd(r_f))
    #         + ")$"
    #     )
    #     row += (
    #         f"&$"
    #         + format_num(np.nanmean(r_f_r))
    #         + "\,\,(\pm "
    #         + format_num(np.nanstd(r_f_r))
    #         + ")$"
    #     )
    #     row += (
    #         f"&$"
    #         + format_num(np.nanmean(tr_f))
    #         + "\,\,(\pm "
    #         + format_num(np.nanstd(tr_f))
    #         + ")$"
    #     )
    #     row += (
    #         f"&$"
    #         + format_num(np.nanmean(tr_f_r))
    #         + "\,\,(\pm "
    #         + format_num(np.nanstd(tr_f_r))
    #         + ")$"
    #     )
    # else:
    #     pass
    #     # row += "-&"+ (
    #     #     f"&$"
    #     #     + format_num(np.nanmean(r_f))
    #     #     + "\,\,(\pm "
    #     #     + format_num(np.nanstd(r_f))
    #     #     + ")$"
    #     # ) +"-&-&-&-"
    # print(row )#+ "\\\\")
    

In [ ]:
from figs.scripts.loader import Loader
from figs.scripts.tables import Tables
from figs.scripts.figures import Figures
from imports.general import *
from imports.ml import *

tables = Tables()
figures = Figures()
# # For paper:
# Tabel 1:
tables.table_linear_correlation()
# Tabel 2:
tables.table_linear_model(X_bo=False)
# Tabel 3:
tables.table_linear_model(X_bo=True)
# Figure 1:
figures.figure_regret_calibration(settings_x = {"bo": True, "metric": "f_regret"},
        settings_y = {"bo": True, "metric": "y_calibration_mse"},
        x_figsettings= {"label": r"$\mathcal{R}_I(f)$", "log": True},
        y_figsettings= {"label": r"$\mathcal{C}_{BO}(y)$", "log": True},)

figures.figure_regret_calibration(settings_x = {"bo": True, "metric": "f_regret"},
        settings_y = {"bo": False, "metric": "y_calibration_mse"},
        x_figsettings= {"label": r"$\mathcal{R}_I(f)$", "log": True},
        y_figsettings= {"label": r"$\mathcal{C}_{R}(y)$", "log": True},)

# 2. Change in predictive std

In [1]:
from figs.scripts.figures import Figures
figures = Figures()
# figures.figure_std_vs_metric(settings={
#             "data_name": "benchmark",
#             "epoch": 90,
#             "snr": 100,
#             "bo": False,
#         })
figures.figure_std_vs_metric(settings={
            "data_name": "benchmark",
            "epoch": 90,
            "snr": 100,
            "bo": True,
        })
figures.figure_std_vs_metric(settings={
            "data_name": "benchmark",
            "epoch": 90,
            "snr": 100,
            "bo": True,
        },y = "f_regret")
figures.scatter_regret_calibration_std_change(average=True)


SELECT y_calibration_mse FROM results_change_std WHERE data_name='benchmark' and epoch='90' and snr='100' and bo='0' and surrogate='BNN' and std_change='0.01';
SELECT y_calibration_mse FROM results_change_std WHERE data_name='benchmark' and epoch='90' and snr='100' and bo='0' and surrogate='BNN' and std_change='0.1';
SELECT y_calibration_mse FROM results_change_std WHERE data_name='benchmark' and epoch='90' and snr='100' and bo='0' and surrogate='BNN' and std_change='0.1414';


AttributeError: 'int' object has no attribute 'dtype'

<Figure size 720x432 with 0 Axes>

In [ ]:
print(np.geomspace(0.1,1.2, 21))


# 3. The effect of the number of samples

In [ ]:
from imports.general import *
from imports.ml import *
from experiments.samples import SamplesExperiment
from scipy.special import erf,erfinv
# exp = SamplesExperiment()
# exp.run()

def g(p,v, mu_t=None):
	if mu_t is None:
		return 1/2*(1 + erf(v*erfinv(2*p-1)))
	else:
		return 1/2*(1 + erf(mu_t/np.sqrt(2) + v*erfinv(2*p-1)))

def g_approx(p,v):	
	x = 2*p - 1
	return p*x + 1/3*(v-v**3)*x + 1/15*v*(2*v**4 - 5*v**2 + 3)*x**5


p = np.linspace(0.001,1,100)
v = np.linspace(0.1,2,10)
colors = plt.cm.plasma(np.linspace(0, 1, len(v)))

plt.figure()
plt.title("Correct mean")
plt.plot(p,p,"--")
for i_v,v_ in enumerate(v):
	gs = []
	g_appoxs = []
	for p_ in p:
		gs.append(g(p_,v_))
		g_appoxs.append(g_approx(p_,v_))
	if i_v == len(v)-1 or i_v == 0:
		plt.plot(p,gs,color=colors[i_v],label=v_)
	else:
		plt.plot(p,gs,color=colors[i_v])
	# plt.plot(p,g_appoxs,"--",color=colors[i_v])
plt.xlabel("Expected percentiles (p)")
plt.ylabel("Observed percentiles")
plt.legend()

for mu in [-2.0,2.0]:
	plt.figure()
	plt.plot(p,p,"--")
	plt.title(f"Mean:{mu}")
	for i_v,v_ in enumerate(v):
		gs = []
		for p_ in p:
			gs.append(g(p_,v_,mu_t=mu))
		if i_v == len(v)-1 or i_v == 0:
			plt.plot(p,gs,color=colors[i_v],label=v_)
		else:
			plt.plot(p,gs,color=colors[i_v])
	plt.legend()
	plt.xlabel("Expected percentiles (p)")
	plt.ylabel("Observed percentiles")



# 4. Recalibration

In [ ]:
from src.parameters import Parameters
from src.recalibrator import Recalibrator
from src.dataset import Dataset
from src.metrics import Metrics
from surrogates.gaussian_process import GaussianProcess

parameters = Parameters({"surrogate":"RF",
"experiment":"1",
"acquisition":"EI",
"data_name":"benchmark",
"n_evals":50,
"n_test":1000,
"snr":10,
"xi":0.0,
"bo":True,
"problem_idx":3, 
"d":1,
"seed":0 ,
"n_initial":10},mkdir=True)

dataset = Dataset(parameters)
model = GaussianProcess(parameters,dataset)
recalibrator = Recalibrator(dataset,model,mode="iid")

model.fit(dataset.data.X_train,dataset.data.y_train)

idx = np.argsort(dataset.data.X_test.squeeze())
X_test = dataset.data.X_test[idx]
y_test = dataset.data.y_test[idx]
mus_,sigmas_ = model.predict(X_test)
mus_ = mus_.squeeze()
sigmas_ = sigmas_.squeeze()
fig = plt.figure()
plt.plot(X_test, mus_, marker=".")
plt.fill_between(X_test.squeeze(), mus_-2*sigmas_,mus_+2*sigmas_,alpha=0.2)
plt.plot(X_test, y_test,"x",alpha=0.2)
plt.plot(dataset.data.X_train, dataset.data.y_train,"o")

mus,sigmas = recalibrator.recalibrate(mus_,sigmas_)
fig = plt.figure()
plt.plot(X_test, mus, marker=".")
plt.fill_between(X_test.squeeze(), mus-2*sigmas,mus+2*sigmas,alpha=0.2)
plt.plot(X_test, y_test,"x",alpha=0.2)
plt.plot(dataset.data.X_train, dataset.data.y_train,"o")

metrics = Metrics(parameters)

metrics.calibration_y_batched(mus_,sigmas_,y_test)
y_calibration_bef = metrics.summary["y_calibration"]

metrics.calibration_y_batched(mus,sigmas,y_test)
y_calibration_aft = metrics.summary["y_calibration"]

fig = plt.figure()
plt.plot(metrics.summary["p_array"],metrics.summary["p_array"],"--")
plt.plot(metrics.summary["p_array"],y_calibration_bef,"-x",label="Before")
plt.plot(metrics.summary["p_array"],y_calibration_aft,"-s",label="After")
plt.legend(); plt.xlabel(r"Expected Confidence Interval"); plt.ylabel("Observed Confidence Interval");



# 4. New loader

In [ ]:
def format_num(x: float,n: int):
	if n == 2:
		return f"{x:.2f}"
	if n == 3:
		return f"{x:.3f}"
	if n == 4:
		return f"{x:.4f}"

cnx = sqlite3.connect('./results.db')
table_exists = cnx.execute(
		f"SELECT name FROM sqlite_master WHERE type='table' AND name='{loaddir}';"
)

y = "f_regret"
for surrogate in ["BNN","GP","RF","DE"]:
	df = pd.read_sql(f"select {y} from results_recalibrated_regret_vs_calibration where surrogate='{surrogate}'", cnx)
	mu, std = format_num(df.mean().to_numpy().squeeze(),n=4),format_num(df.std().to_numpy().squeeze(),n=2)
	print(surrogate,"calibrated",f"${mu}\,\,(\pm {std})$")
	df = pd.read_sql(f"select {y} from results_regret_vs_calibration where surrogate='{surrogate}' and bo='1'", cnx)
	mu, std = format_num(df.mean().to_numpy().squeeze(),n=4),format_num(df.std().to_numpy().squeeze(),n=2)
	print(surrogate,"not calibrated",f"${mu}\,\,(\pm {std})$")

In [ ]:
# python scipy ks test:
# akkumuleret/CDF fordeling fra ét y vs. empirisk akkumuleret fordeling